<a href="https://colab.research.google.com/github/tataxverse/tensorflow-specialisation-deeplearning.ai/blob/main/distributed-training/mirrored_strategy_w4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Import TensorFlow and TensorFlow Datasets

import tensorflow_datasets as tfds
import tensorflow as tf
tfds.disable_progress_bar()

import os

In [4]:
# Load the dataset we'll use for this lab
datasets, info = tfds.load(name='mnist', with_info=True, as_supervised=True, data_dir='./data')

mnist_train, mnist_test = datasets['train'], datasets['test']

Dataset mnist downloaded and prepared to data/mnist/3.0.1. Subsequent calls will reuse this data.


Next, you define strategy using the MirroredStrategy() class. Print to see the number of devices available.

In [5]:
strategy=tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

Number of devices: 1


In [8]:
num_train_examples=info.splits['train'].num_examples
num_test_examples=info.splits['test'].num_examples

BUFFER_SIZE=10000

BATCH_SIZE_PER_REPLICA=64 #the distribution you are making for each available device.
BATCH_SIZE=BATCH_SIZE_PER_REPLICA*strategy.num_replicas_in_sync

In [9]:
def scale(image,label):
  image=tf.cast(image,tf.float32)
  image/=255
  return image,label

In [10]:
train_dataset=mnist_train.map(scale).cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
eval_dataset=mnist_test.map(scale).batch(BATCH_SIZE)

For your model to follow the strategy, define your model within the strategy's scope.

 The important thing to notice and compare is the time taken for each epoch to complete.

In [14]:
with strategy.scope():
  model=tf.keras.Sequential([
      tf.keras.layers.Conv2D(32,3,activation='relu',input_shape=(28,28,1)),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(64,activation='relu'),
      tf.keras.layers.Dense(10,activation='softmax')
  ])
  model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [15]:
model.fit(train_dataset,epochs=12)

Epoch 1/12


/usr/local/lib/python3.11/dist-packages/keras/src/backend/tensorflow/nn.py:708: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


938/938 ━━━━━━━━━━━━━━━━━━━━ 40s 40ms/step - accuracy: 0.8876 - loss: 0.3901
Epoch 2/12
938/938 ━━━━━━━━━━━━━━━━━━━━ 30s 32ms/step - accuracy: 0.9780 - loss: 0.0745
Epoch 3/12
938/938 ━━━━━━━━━━━━━━━━━━━━ 31s 33ms/step - accuracy: 0.9849 - loss: 0.0504
Epoch 4/12
938/938 ━━━━━━━━━━━━━━━━━━━━ 31s 33ms/step - accuracy: 0.9880 - loss: 0.0388
Epoch 5/12
938/938 ━━━━━━━━━━━━━━━━━━━━ 29s 31ms/step - accuracy: 0.9919 - loss: 0.0263
Epoch 6/12
938/938 ━━━━━━━━━━━━━━━━━━━━ 31s 33ms/step - accuracy: 0.9936 - loss: 0.0209
Epoch 7/12
938/938 ━━━━━━━━━━━━━━━━━━━━ 30s 32ms/step - accuracy: 0.9953 - loss: 0.0158
Epoch 8/12
938/938 ━━━━━━━━━━━━━━━━━━━━ 41s 32ms/step - accuracy: 0.9962 - loss: 0.0118
Epoch 9/12
938/938 ━━━━━━━━━━━━━━━━━━━━ 31s 33ms/step - accuracy: 0.9967 - loss: 0.0104
Epoch 10/12
938/938 ━━━━━━━━━━━━━━━━━━━━ 43s 34ms/step - accuracy: 0.9971 - loss: 0.0087
Epoch 11/12
938/938 ━━━━━━━━━━━━━━━━━━━━ 31s 33ms/step - accuracy: 0.9980 - loss: 0.0057
Epoch 12/12
938/938 ━━━━━━━━━━━━━━━━━━━━ 